## 解答

下面的代码用自制的Cholesky分解和scipy库给出的Cholesky分解，分别求解了一系列和Hilbert矩阵有关的问题。有以下发现：

- 我的实现在测试的例子中比scipy的库函数误差更小
- Hilbert矩阵非常病态，十分接近奇异矩阵。具体来说，它的cond值特别大，与n成指数关系，这导致了：
  - H_14被两种算法都判定为奇异矩阵
  - 求解 Hx=b，b的微小差异对x的影响极大
  - 这些性质直接导致 scipy 中的相关函数报错

在 https://mathoverflow.net/questions/137059/the-singular-values-of-the-hilbert-matrix 记载了一些关于 Hilbert 矩阵奇异性的渐进分析可供参考。Matlab官方推荐使用符号计算处理 Hilbert 矩阵相关问题。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def cholesky(A: np.array):
    n = A.shape[0]
    for j in range(n):
        for k in range(j):
            A[j, j] -= A[j, k] ** 2
        A[j, j] = np.sqrt(A[j, j])
        for i in range(j + 1, n):
            for k in range(j):
                A[i, j] -= A[i, k] * A[j, k]
            A[i, j] /= A[j, j]
    return A
def solve_Lx_b(L: np.array, b: np.array):
    n = L.shape[0]
    x = np.zeros(n)
    for i in range(n):
        x[i] = b[i]
        for j in range(i):
            x[i] -= L[i, j] * x[j]
        x[i] /= L[i, i]
    return x
def solve_Ux_b(U: np.array, b: np.array):
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, n):
            x[i] -= U[i, j] * x[j]
        x[i] /= U[i, i]
    return x

In [3]:
from scipy.linalg import hilbert, solve_triangular, cho_factor, cho_solve
def get_errors(n, error_in_b=None):
    x = np.ones(n)
    H = hilbert(n)
    b = H @ x
    if error_in_b:
        b += np.random.uniform(-1, 1, n) * error_in_b
    L = cholesky(H)
    U = L.T
    y_hat = solve_Lx_b(L, b)
    x_hat = solve_Ux_b(U, y_hat)
    return np.linalg.norm(x_hat - x, ord=np.inf), np.linalg.norm(b - H @ x_hat, ord=np.inf)

def get_errors2(n, error_in_b=None):
    x = np.ones(n)
    H = hilbert(n)
    b = H @ x
    if error_in_b:
        b += np.random.uniform(-1, 1, n) * error_in_b
    factor = cho_factor(H, lower=True, overwrite_a=True)
    x_hat = cho_solve(factor, b)
    return np.linalg.norm(x_hat - x, ord=np.inf), np.linalg.norm(b - H @ x_hat, ord=np.inf)

print(get_errors(10))
print(get_errors(10, 1e-7))
print(get_errors(8))
print(get_errors(12))
print(get_errors(14))

print(get_errors2(10))
print(get_errors2(10, 1e-7))
print(get_errors2(8))
print(get_errors2(12))
print(get_errors2(14))


(0.0006437815559028337, 0.2211167902289246)
(53974.107015939335, 2249.0904516247733)
(6.271987497141751e-07, 0.196982595825971)
(0.355545598229285, 0.24029154364420474)
(nan, nan)
(0.0009051867107066069, 2.220446049250313e-16)
(187136.81737277217, 1.106048586052566e-11)
(7.844380782717764e-07, 2.220446049250313e-16)
(0.5792093276753197, 2.220446049250313e-16)


/tmp/ipykernel_64768/1229710670.py:6: RuntimeWarning: invalid value encountered in sqrt
  A[j, j] = np.sqrt(A[j, j])


LinAlgError: 14-th leading minor of the array is not positive definite